In [56]:
import torchreid

## Image dataset - market1501

In [57]:
height=256
width = 128
datamanager = torchreid.data.ImageDataManager(
    root="reid-data",
    sources="market1501",
    targets="market1501",
    height=height,
    width=width,
    batch_size_train=32,
    batch_size_test=100,
    transforms=["random_flip", "random_crop"]
)

# return train loader of source data
train_loader = datamanager.train_loader

# return test loader of target data
test_loader = datamanager.test_loader

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gal

In [58]:
from torchreid import metrics

# Returns query and gallery of a test dataset, each containing tuples of (img_path(s), pid, camid)
dataset = datamanager.fetch_test_loaders("market1501")
data_type = "image"
sav_dir = "/output"


### Define hyperparameter

In [27]:
'''
load model
Use `torchreid.models.show_avai_models()` to check available model
''' 
torchreid.models.show_avai_models()

#create model
model = torchreid.models.build_model(
    name="osnet_x1_0",
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)
model = model.cuda()

# load pretrained weight to the model
weight_path = 'log/osnet_x1_0_market_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth'
torchreid.utils.load_pretrained_weights(model, weight_path)

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

Successfully loaded imagenet pretrained weights from "C:\Users\Drayang/.cache\torch\checkpoints\osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Successfully loaded pretrained weights from "log/osnet_x1_0_market_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth"


### Compile the model

In [28]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

### Train and test the model

In [31]:
engine.run(
    save_dir="log/osnet_x1_0",
    max_epoch=60,
    eval_freq=10,
    print_freq=10,
    test_only=True,
    visrank = True, # visualize rank result
    visrank_topk = 5 # image to save
)

##### Evaluating market1501 (source) #####
Extracting features from query set ...
Done, obtained 3368-by-512 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-512 matrix
Speed: 0.0338 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 82.6%
CMC curve
Rank-1  : 94.3%
Rank-5  : 97.9%
Rank-10 : 98.6%
Rank-20 : 99.2%
# query: 3368
# gallery 15913
Visualizing top-5 ranks ...
- done 100/3368
- done 200/3368
- done 300/3368
- done 400/3368
- done 500/3368
- done 600/3368
- done 700/3368
- done 800/3368
- done 900/3368
- done 1000/3368
- done 1100/3368
- done 1200/3368
- done 1300/3368
- done 1400/3368
- done 1500/3368
- done 1600/3368
- done 1700/3368
- done 1800/3368
- done 1900/3368
- done 2000/3368
- done 2100/3368
- done 2200/3368
- done 2300/3368
- done 2400/3368
- done 2500/3368
- done 2600/3368
- done 2700/3368
- done 2800/3368
- done 2900/3368
- done 3000/3368
- done 3100/3368
- done 3200/3368
- done 3300/33

## Video Dataset - prid-2011


In [32]:
datamanager = torchreid.data.VideoDataManager(
    root='reid-data',
    sources='prid2011',
    height=256,
    width=128,
    batch_size_train=3,
    batch_size_test=3,
    seq_len=15,
    sample_method='evenly'
)

# return train loader of source data
train_loader = datamanager.train_loader

# return test loader of target data
test_loader = datamanager.test_loader

Building train transforms ...
+ resize to 256x128
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded PRID2011
  -------------------------------------------
  subset   | # ids | # tracklets | # cameras
  -------------------------------------------
  train    |    89 |         178 |         2
  query    |    89 |          89 |         1
  gallery  |    89 |          89 |         1
  -------------------------------------------
=> Loading test (target) dataset
=> Loaded PRID2011
  -------------------------------------------
  subset   | # ids | # tracklets | # cameras
  -------------------------------------------
  train    |    89 |         178 |         2
  query    |    89 |          89 |         1
  gallery  |    89 |     

In [36]:


#create model
model = torchreid.models.build_model(
    name="osnet_x1_0",
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)
model = model.cuda()

# load pretrained weight to the model
weight_path = 'log/osnet_x1_0_market_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth'
torchreid.utils.load_pretrained_weights(model, weight_path)



optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0003
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)


Successfully loaded imagenet pretrained weights from "C:\Users\Drayang/.cache\torch\checkpoints\osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Successfully loaded pretrained weights from "log/osnet_x1_0_market_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [37]:
# COmpile model
engine = torchreid.engine.VideoSoftmaxEngine(
    datamanager, model, optimizer, scheduler=scheduler,
    pooling_method='avg'
)


In [39]:
engine.run(
    max_epoch=60,
    save_dir='log/oxnet_prid2011',
    print_freq=10,
    test_only=True,
#     visrank = True, # visualize rank result
#     visrank_topk = 5 # image to save
)

##### Evaluating prid2011 (source) #####
Extracting features from query set ...
Done, obtained 89-by-512 matrix
Extracting features from gallery set ...
Done, obtained 89-by-512 matrix
Speed: 0.0315 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 42.2%
CMC curve
Rank-1  : 30.3%
Rank-5  : 52.8%
Rank-10 : 60.7%
Rank-20 : 78.7%


## Video dataset - dukemtmc-vidreid

In [48]:
datamanager = torchreid.data.VideoDataManager(
    root='reid-data',
    sources='dukemtmcvidreid',
    height=256,
    width=128,
    batch_size_train=10,
    batch_size_test=10,
    seq_len=15,
    sample_method='evenly'
)

# return train loader of source data
train_loader = datamanager.train_loader

# return test loader of target data
test_loader = datamanager.test_loader

Building train transforms ...
+ resize to 256x128
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Generating split json file (** this might take a while **)
Processing "C:\Users\Drayang\GitHub\deep-person-reid\reid-data\dukemtmc-vidreid\DukeMTMC-VideoReID/train" with 702 person identities
Saving split to C:\Users\Drayang\GitHub\deep-person-reid\reid-data\dukemtmc-vidreid\split_train.json
=> Generating split json file (** this might take a while **)
Processing "C:\Users\Drayang\GitHub\deep-person-reid\reid-data\dukemtmc-vidreid\DukeMTMC-VideoReID/query" with 702 person identities
Saving split to C:\Users\Drayang\GitHub\deep-person-reid\reid-data\dukemtmc-vidreid\split_query.json
=> Generating split json file (** this might take

C:\Users\Drayang\GitHub\deep-person-reid\torchreid\data\datasets\video\dukemtmcvidreid.py:107: UserWarning: Index name F0001 in C:\Users\Drayang\GitHub\deep-person-reid\reid-data\dukemtmc-vidreid\DukeMTMC-VideoReID/gallery\0002\2197 is missing, skip
  warnings.warn(


Saving split to C:\Users\Drayang\GitHub\deep-person-reid\reid-data\dukemtmc-vidreid\split_gallery.json
=> Loaded DukeMTMCVidReID
  -------------------------------------------
  subset   | # ids | # tracklets | # cameras
  -------------------------------------------
  train    |   702 |        2196 |         8
  query    |   702 |         702 |         8
  gallery  |  1110 |        2636 |         8
  -------------------------------------------
=> Loading test (target) dataset
=> Loaded DukeMTMCVidReID
  -------------------------------------------
  subset   | # ids | # tracklets | # cameras
  -------------------------------------------
  train    |   702 |        2196 |         8
  query    |   702 |         702 |         8
  gallery  |  1110 |        2636 |         8
  -------------------------------------------


  **************** Summary ****************
  source             : ['dukemtmcvidreid']
  # source datasets  : 1
  # source ids       : 702
  # source tracklets : 2196
  # sou

In [49]:
#create model
model = torchreid.models.build_model(
    name="osnet_x1_0",
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)
model = model.cuda()

# load pretrained weight to the model
weight_path = 'log/osnet_x1_0_duke_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth'
torchreid.utils.load_pretrained_weights(model, weight_path)



optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0003
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)


Successfully loaded imagenet pretrained weights from "C:\Users\Drayang/.cache\torch\checkpoints\osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Successfully loaded pretrained weights from "log/osnet_x1_0_duke_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth"


In [54]:
# COmpile model
engine = torchreid.engine.VideoSoftmaxEngine(
    datamanager, model, optimizer, scheduler=scheduler,
    pooling_method='avg'
)

engine.run(
    max_epoch=20,
    save_dir='log/oxnet_ilidsvid',
    print_freq=10,
    test_only=True,
#     visrank = True, # visualize rank result
#     visrank_topk = 5 # image to save
)

##### Evaluating dukemtmcvidreid (source) #####
Extracting features from query set ...
Done, obtained 702-by-512 matrix
Extracting features from gallery set ...
Done, obtained 2636-by-512 matrix
Speed: 0.0347 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 91.1%
CMC curve
Rank-1  : 92.6%
Rank-5  : 98.6%
Rank-10 : 99.1%
Rank-20 : 99.4%


## Video Dataset- iLDS-VID

In [47]:
datamanager = torchreid.data.VideoDataManager(
    root='reid-data',
    sources='ilidsvid',
    height=256,
    width=128,
    batch_size_train=3,
    batch_size_test=3,
    seq_len=15,
    sample_method='evenly'
)

# return train loader of source data
train_loader = datamanager.train_loader

# return test loader of target data
test_loader = datamanager.test_loader

Building train transforms ...
+ resize to 256x128
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded iLIDSVID
  -------------------------------------------
  subset   | # ids | # tracklets | # cameras
  -------------------------------------------
  train    |   150 |         300 |         2
  query    |   150 |         150 |         1
  gallery  |   150 |         150 |         1
  -------------------------------------------
=> Loading test (target) dataset
=> Loaded iLIDSVID
  -------------------------------------------
  subset   | # ids | # tracklets | # cameras
  -------------------------------------------
  train    |   150 |         300 |         2
  query    |   150 |         150 |         1
  gallery  |   150 |     

### using the same model and hyperparameter as previous


In [42]:
# COmpile model
engine = torchreid.engine.VideoSoftmaxEngine(
    datamanager, model, optimizer, scheduler=scheduler,
    pooling_method='avg'
)

engine.run(
    max_epoch=60,
    save_dir='log/oxnet_ilidsvid',
    print_freq=10,
    test_only=True,
#     visrank = True, # visualize rank result
#     visrank_topk = 5 # image to save
)

##### Evaluating ilidsvid (source) #####
Extracting features from query set ...
Done, obtained 150-by-512 matrix
Extracting features from gallery set ...
Done, obtained 150-by-512 matrix
Speed: 0.0319 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 22.0%
CMC curve
Rank-1  : 14.7%
Rank-5  : 26.7%
Rank-10 : 32.7%
Rank-20 : 42.0%
